# Analyzing CIA Factbook Data Using SQL

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

ModuleNotFoundError: No module named 'sql'

In [4]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

UsageError: Cell magic `%%sql` not found.


In [ ]:
%%sql
SELECT * 
    FROM facts LIMIT 5;

Minimum population
Maximum population
Minimum population growth
Maximum population growth

In [ ]:
%%sql
SELECT MIN(population) AS Min_pop, MAX(population) AS Max_pop, MIN(population_growth) AS Min_growth, MAX (population_growth) AS Max_growth
    FROM facts;

Countries with the minimum population.

In [ ]:
%%sql
SELECT name AS Country
    FROM facts
WHERE population = (SELECT MIN(population) FROM facts);

Countries with the maximum population.

In [ ]:
%%sql
SELECT name AS Country
    FROM facts
WHERE population = (SELECT MAX(population) FROM facts);

Without Outliers (Word / Antarctica)
Minimum population
Maximum population
Minimum population growth
Maximum population growth

In [ ]:
%%sql
SELECT MIN(population) AS Min_pop, MAX(population) AS Max_pop, MIN(population_growth) AS Min_growth, MAX (population_growth) AS Max_growth
    FROM facts
WHERE name <> 'World' AND name <> "Antarctica";

Average value for the following columns:

population
area

In [ ]:
%%sql
SELECT CAST(AVG(population) AS INT) AS Mean_pop, CAST(AVG(area) AS INT) AS Mean_area
    FROM facts
WHERE name <> 'World' AND name <> "Antarctica";

countries meeting both of the following criteria:
The population is above average.
The area is below average.

In [ ]:
%%sql

SELECT name AS 'Country with above averga pop. / area'
    FROM facts
WHERE population > (SELECT AVG(population) FROM facts)
AND area > (SELECT AVG(area) FROM facts);

 10M population capital cities ranked from biggest to smallest population

In [5]:
%%sql

SELECT c.name AS capital_city, f.name AS country, c.population AS population
    FROM facts AS f
INNER JOIN
(SELECT * FROM cities WHERE capital = 1 AND population > 10000000)  AS c
    ON f.id = c.facts_id
ORDER BY 3 DESC;

UsageError: Cell magic `%%sql` not found.


Query to find the countries where the urban center (city) population is more than half of the country's total population

In [ ]:
%%sql

SELECT f.name AS country, (SELECT SUM(c.sum_pop) FROM cities) AS urban_pop, f.population AS total_pop, ROUND(CAST(c.sum_pop AS FLOAT)/f.population, 6) AS urban_pct
FROM facts AS f
    INNER JOIN (SELECT facts_id, SUM(population) AS sum_pop FROM cities GROUP BY facts_id) AS c
    ON c.facts_id = f.id
WHERE urban_pct > 0.5   
GROUP BY country    
ORDER BY urban_pct ASC;